## Import modules

In [82]:
import pandas as pd
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

## Read data


In [83]:
train_data = pd.read_csv("./Breast_cancer_data.csv")
train_data

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0
...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0
565,20.13,28.25,131.20,1261.0,0.09780,0
566,16.60,28.08,108.30,858.1,0.08455,0
567,20.60,29.33,140.10,1265.0,0.11780,0


In [84]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mean_radius      569 non-null    float64
 1   mean_texture     569 non-null    float64
 2   mean_perimeter   569 non-null    float64
 3   mean_area        569 non-null    float64
 4   mean_smoothness  569 non-null    float64
 5   diagnosis        569 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 26.8 KB


In [85]:
train_data.diagnosis.value_counts()

1    357
0    212
Name: diagnosis, dtype: int64

In [86]:
x_data = train_data.drop(['diagnosis'], axis=1)
y_data = train_data.diagnosis

In [87]:
from sklearn.model_selection import train_test_split

In [88]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=0)

## Train

In [89]:
lgb_clf = LGBMClassifier()
xgb_clf = XGBClassifier()

In [90]:
lgb_clf.fit(x_train, y_train)

LGBMClassifier()

In [91]:
xgb_clf.fit(x_train, y_train)

[23:33:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
/home/sokol/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## Evaluate

In [92]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import accuracy_score, roc_auc_score

In [93]:
y_pred = lgb_clf.predict(x_test)
print('Accuracy', accuracy_score(y_test, y_pred))
print('F1', f1_score(y_test, y_pred).astype(float))
print('Precision', precision_score(y_test, y_pred))
print('Recall', recall_score(y_test, y_pred).astype(float))

Accuracy 0.9298245614035088
F1 0.9454545454545454
Precision 0.9285714285714286
Recall 0.9629629629629629


In [94]:
y_pred = xgb_clf.predict(x_test)
print('Accuracy', accuracy_score(y_test, y_pred))
print('F1', f1_score(y_test, y_pred).astype(float))
print('Precision', precision_score(y_test, y_pred))
print('Recall', recall_score(y_test, y_pred).astype(float))

Accuracy 0.935672514619883
F1 0.9497716894977168
Precision 0.9369369369369369
Recall 0.9629629629629629
/home/sokol/.local/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [96]:
xgb_clf.save_model('xgboost_clf_model.json')
lgb_clf.booster_.save_model('lightgbm_clf_model.txt')